In [39]:
from bpemb import BPEmb
from tqdm import tqdm
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
bpemb_en=BPEmb(lang="en",dim=50)

In [48]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dropout, Conv1D, GlobalMaxPooling1D, Dense, Activation

In [179]:
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
print(bpemb_en.encode("Stratford"))
print(bpemb_en.embed("Stratford").shape)

['▁strat', 'ford']
(2, 50)


In [150]:
max_len = 0

In [154]:
with open("datasets/Chatbot/train.csv") as f:
    reader = csv.reader(f, delimiter='\t')
    max_len = 0
    y = []
    for row in tqdm(reader):
        y.append(row[1])
        sample_len = len(bpemb_en.encode(row[0]))
        max_len = sample_len if sample_len > max_len else max_len


100it [00:00, 33498.16it/s]


In [155]:
print(max_len)
print(y[:10])

32
['FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection']


In [26]:
max_len = max_len + 1

In [27]:
# label encoder
le = LabelEncoder()
encoded_labels = le.fit_transform(y)
#print(encoded_labels)
print(le.classes_)

['DepartureTime' 'FindConnection']


In [28]:
y = to_categorical(encoded_labels, num_classes=len(le.classes_))0

In [29]:
x = None

In [127]:
with open("datasets/Chatbot/train.csv") as f:
        reader = csv.reader(f, delimiter='\t')
        x = None
        
        for row in tqdm(reader):            
            # features
            embeddings = bpemb_en.embed(row[0])
            padding_vec = np.zeros((max_len - embeddings.shape[0], 50))
            padded = np.vstack((embeddings, padding_vec))
            padded = np.expand_dims(padded, axis=0)
            #print(padded.shape)
            if x is not None:
                x = np.vstack((x, padded))
            else:
                x = padded

100it [00:00, 4180.18it/s]


In [119]:
def file_to_x(filename="datasets/Chatbot/train.csv", max_len=max_len):
    max_len = max_len+1
    
    with open(filename) as f:
        reader = csv.reader(f, delimiter='\t')
        y = []
        for row in tqdm(reader):
            y.append(row[1])
        y = to_categorical(encoded_labels, num_classes=len(le.classes_))
        
    with open(filename) as f:
        reader = csv.reader(f, delimiter='\t')
        x = None
        
        for row in tqdm(reader):            
            # features
            embeddings = bpemb_en.embed(row[0])
            padding_vec = np.zeros((max_len - embeddings.shape[0], 50))
            padded = np.vstack((embeddings, padding_vec))
            padded = np.expand_dims(padded, axis=0)
            #print(padded.shape)
            if x is not None:
                x = np.vstack((x, padded))
            else:
                x = padded
                
    

            
            
    return x, y

In [31]:
x.shape

(100, 33, 50)

In [53]:
batch_size = 32
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 200

In [35]:
print("Bulding the Neural Network")

Bulding the Neural Network


In [57]:
model = Sequential()

# Dropping 20% nodes from the embedding layers
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(len(le.classes_)))
model.add(Activation('sigmoid'))

# compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [58]:
model.fit(x, y,
          batch_size=batch_size,
          epochs=20,
          validation_split=0.1)

Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - ETA: 1s - loss: 0.7993 - acc: 0.468 - 1s 8ms/step - loss: 0.7136 - acc: 0.5167 - val_loss: 0.6190 - val_acc: 0.7500
Epoch 2/20
90/90 [==============================] - ETA: 0s - loss: 0.6983 - acc: 0.500 - 0s 211us/step - loss: 0.6317 - acc: 0.6333 - val_loss: 0.5802 - val_acc: 0.8000
Epoch 3/20
90/90 [==============================] - ETA: 0s - loss: 0.5644 - acc: 0.750 - 0s 233us/step - loss: 0.5434 - acc: 0.7500 - val_loss: 0.5177 - val_acc: 0.9000
Epoch 4/20
90/90 [==============================] - ETA: 0s - loss: 0.4882 - acc: 0.875 - 0s 211us/step - loss: 0.4716 - acc: 0.8333 - val_loss: 0.4431 - val_acc: 0.9000
Epoch 5/20
90/90 [==============================] - ETA: 0s - loss: 0.4531 - acc: 0.828 - 0s 222us/step - loss: 0.4147 - acc: 0.8556 - val_loss: 0.3617 - val_acc: 0.9500
Epoch 6/20
90/90 [==============================] - ETA: 0s - loss: 0.3444 - acc: 0.921 - 0s 211us/step - lo

In [120]:
x_test, y_test = file_to_x(filename="datasets/Chatbot/test.csv")

106it [00:00, 106286.45it/s]
106it [00:00, 3538.96it/s]


In [121]:
pred = model.predict_classes(x_test)

In [125]:
len(x_test)

106

In [122]:
pred_cat = to_categorical(pred)

In [123]:
len(pred_cat)

106

In [156]:
max_len = max_len + 1

In [157]:
with open("datasets/Chatbot/test.csv") as f:
    reader = csv.reader(f, delimiter='\t')
    
    y_test = []
    for row in tqdm(reader):
        y_test.append(row[1])

106it [00:00, 108969.66it/s]


In [170]:
y_test = to_categorical(le.transform(y_test))

In [158]:
max_len

33

In [159]:
with open("datasets/Chatbot/test.csv") as f:
        reader = csv.reader(f, delimiter='\t')
        x_test = None
        
        for row in tqdm(reader):            
            # features
            embeddings = bpemb_en.embed(row[0])
            padding_vec = np.zeros((max_len - embeddings.shape[0], 50))
            padded = np.vstack((embeddings, padding_vec))
            padded = np.expand_dims(padded, axis=0)
            #print(padded.shape)
            if x_test is not None:
                x_test = np.vstack((x_test, padded))
            else:
                x_test = padded

106it [00:00, 4082.95it/s]


In [171]:
pred = model.predict_classes(x_test)
pred_cat = to_categorical(pred)

In [180]:
confusion_matrix(np.argmax(y_test, axis=1), np.argmax(pred_cat, axis=1))

array([[34,  1],
       [ 1, 70]], dtype=int64)

In [182]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_cat, axis=1)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        35
           1       0.99      0.99      0.99        71

   micro avg       0.98      0.98      0.98       106
   macro avg       0.98      0.98      0.98       106
weighted avg       0.98      0.98      0.98       106

